In [2]:
!pip install -q langchain_community langchain_huggingface faiss-gpu rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.3 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import torch
from langchain.retrievers import BM25Retriever
import numpy as np
from langchain.llms import ChatGLM

In [65]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
import pandas as pd

df = pd.read_csv('/content/ds queans.csv', sep=';')[:-1]
df = df.rename(columns={"Unnamed: 0": 'question', "Unnamed: 1": 'answer'})
df = df[(df['question'].str.len() >= 2) & (df['answer'].str.len() >= 2)]

del df['question']

df.to_csv('df.csv')
df

,answer
0,Обращение от XXXX года неактуально.
1,добрый день! Доработка функционала XC ERP X.X...
2,Доработка функционала возможна в рамках запрос...
3,Добрый день. а. В системе созданы отчеты по де...
4,Доработка функционала возможна в рамках запрос...
...,...
22361,Здравствуйте. Обращение SDXXXXXXXX отозвано
22362,"Здравствуйте, Владимир Олегович! Настроила для..."
22363,Добрый день. а. Очищен регистр рамочных догово...
22364,Здравствуйте. Обращение SDXXXXXXXX отозвано


In [22]:
# Шаг 1: Загрузка и обработка локальных документов
document_loader = CSVLoader(file_path="/content/df.csv")
documents = document_loader.load()

# Шаг 2: Разделение текста на чанки
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [4]:
# Шаг 3: Векторизация текстовых чанков и сохранение в векторном хранилище
device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = HuggingFaceEmbeddings(
    model_name="sberbank-ai/sbert_large_nlu_ru",
    show_progress=True,
    model_kwargs={'device': 'cuda'})

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [13]:
vector_store = FAISS.load_local("/content/faiss weight",embeddings=embedding_model,allow_dangerous_deserialization=True)
# FAISS.embeddings
vector_store

In [86]:
# Функция для поиска релевантных текстов
def retrieve_related_texts(query, top_k=1):
    query_embedding = np.array(embedding_model.embed_query(query)).astype('float32').reshape(1, -1)
    distances, indices = vector_store.index.search(query_embedding, top_k)
    return [text_chunks[i] for i in indices[0]]

In [106]:
# Формирование запроса для генеративной модели

query = str(input())
related_text_chunks = str(retrieve_related_texts(query)[0])[10:-53]


prompt_template = """
Вы, опытный и дружелюбный ассистент. Вот несколько фрагментов текста, которые могут быть полезны для ответа на вопрос пользователя:{}
Пожалуйста, используйте эту информацию, чтобы ответить на следующий вопрос:{}
"""
context = related_text_chunks
prompt = prompt_template.format(context, query)
prompt

привет как дела


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"\nВы, опытный и дружелюбный ассистент. Вот несколько фрагментов текста, которые могут быть полезны для ответа на вопрос пользователя:nt='обращение с описанием этапов согласования по подразделениям и ответственным по ним. Указать какие виды договоров необходимо согласовывать. Коллеги запросят необходимую информацию для настройки маршрутов согласования, если будет недостаточно каких-то данных.  X. Может ли устанавливаться статус ``Действует`` и ``Закрыт`` в списке договоров автоматически в зависимости от даты начала и окончания действия, установленных в договоре? Такой функционал не реализован в системе. Если такая доработка\nПожалуйста, используйте эту информацию, чтобы ответить на следующий вопрос:привет как дела\n"

In [68]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained('sberbank-ai/ruT5-large')

# tokenizer=T5Tokenizer.from_pretrained('sberbank-ai/ruT5-large',)

In [67]:
# input_ids = tokenizer(prompt,
#                       padding=True,
#                       truncation=True,
#                       max_length=250,
#                       return_tensors="pt")["input_ids"]
# # len(input_ids['input_ids'])
# out_ids=model.generate(input_ids=input_ids,
#                         max_length=500,
#                         eos_token_id=tokenizer.eos_token_id)

# tokenizer.decode(out_ids[0][:])

"<pad> <extra_id_0>,'src': '/ <extra_id_1>? <extra_id_2> : <extra_id_3>? <extra_id_4>? <extra_id_5> ] Спасибо. [///csv] <extra_id_6>? <extra_id_7> <extra_id_8> <extra_id_9>.] <extra_id_10> <extra_id_11> ] <extra_id_12> <extra_id_13>? <extra_id_14> <extra_id_15> ] [Document() <extra_id_16> ]] [///csv] [//csv] [//csv] <extra_id_17> ] <extra_id_18> ] <extra_id_19> ] <extra_id_20> ]] [///]]] <extra_id_14> ]] <extra_id_14> ]] <extra_id_14> ]] <extra_id_14> ]</s>"

In [ ]:
context = pd.read_csv

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
text = prompt
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
# context_input_ids = tokenizer.encode(context, return_tensors="pt").cuda()

out = model.generate(input_ids.cuda(),
                     num_beams=5,
                     repetition_penalty=1.2,
                     max_new_tokens=150)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

In [113]:
# gc.collect()
torch.cuda.empty_cache()